1. find flask ids in item_data
2. cross-reference these ids with user_items #skip
3. determine amount of flasks available
4. cross-reference choice id from flasks in item_data with item id in item_data #later?
5. determine amount of potential shards for each champion
6. get current shards for each champion owned
7. get level and necessary amount to upgrade each champion owned
8. determine which champions can be leveled up #and by using which flasks (done before)
9. make exceptions for lev (very optional)
10. turn this into a standalone .py and make it automated with pulling the data when loading kong

In [436]:
# imports
import numpy as np
import pandas as pd
# import xml.etree.ElementTree as ET
from lxml import etree
import re
import itertools

In [437]:
# parse data into data structures

user_items_raw = pd.read_json("user_items.json", orient="index")
# print(user_items_raw.head())

item_data_raw = pd.read_json("item_data.json", orient="index")
# print(item_data_raw.head())

user_units_raw = pd.read_json("user_units.json", orient="index")
# print(user_units_raw.head())

# use ET for cards_shard.xml
cards_shard = etree.parse("cards_shard.xml") # type: ignore
root = cards_shard.getroot()

cards_shard_dict = {}
for unit in root.findall("unit"):
    cards_shard_dict[int(unit.find("id").text)] = unit.find("name").text

C:\Users\Felix\AppData\Local\Temp\ipykernel_7252\1498120538.py:3: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  user_items_raw = pd.read_json("user_items.json", orient="index")
C:\Users\Felix\AppData\Local\Temp\ipykernel_7252\1498120538.py:6: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  item_data_raw = pd.read_json("item_data.json", orient="index")
C:\Users\Felix\AppData\Local\Temp\ipykernel_7252\1498120538.py:6: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsin

In [438]:
# constants for id boundaries (04.04.23), easier to change here than in-line 
flasks_id_min = 2534
flasks_id_max = 2679

# reward, aether, chaos, wyld
champion_id_min = [4000, 5600, 6600, 7600]
champion_id_max = [4999, 5999, 6999, 7999]

# shard ids are auto-generated in-game as well
champion_shards_id_min = []
champion_shards_id_max = []
for id_min, id_max in zip(champion_id_min, champion_id_max):
    champion_shards_id_min.append(id_min + 100000)
    champion_shards_id_max.append(id_max + 100000)

# constant champ level requirements
champion_costs = [50, 75, 100, 150, 225, 300, 400, 600, 900, 1200]
champion_costs_mythic = [150, 200, 250, 300, 450, 600, 750, 900, 1100, 1300]

In [439]:
# 1 find flask ids by regex (excluding mythical flasks manually)
item_data_flask_ids = item_data_raw[item_data_raw["name"].str.contains("Flask")][4:] # .dropna(axis=1, how="all")
flasks_id_min_temp = item_data_flask_ids["id"].min()
flasks_id_max_temp = item_data_flask_ids["id"].max()

# prevent reducing the id range
if flasks_id_min_temp <= flasks_id_min and flasks_id_max_temp >= flasks_id_max:
    flasks_id_min = flasks_id_min_temp
    flasks_id_max = flasks_id_max_temp
else:
    print("Error in #1: this action could possibly exclude flasks")
    # return
    
# create new dataframe with flask id and respective reward ids 
rows = []
for index, row in item_data_flask_ids.iterrows():
    obj_id = row['id']
    obj_name = row["name"]
    reward_ids = []
    for choice in row['choice']:
        reward_ids.append(choice['item'][0]['id'])
    rows.append([obj_id, obj_name] + reward_ids)

item_data_flask_rewards = pd.DataFrame(rows, columns=['id', "flask_name", 'reward1', 'reward2', 'reward3', 'reward4'])
item_data_flask_rewards.head()

,id,flask_name,reward1,reward2,reward3,reward4
0,2534,Regal Teroxis Flask,104000,104001,104002,104013
1,2536,Metallic Ralgan Flask,104001,104002,104004,104014
2,2537,Stout Ralgan Flask,104000,104005,104006,104014
3,2538,Vicious Batara Flask,104001,104005,104006,104015
4,2539,Stormy Batara Flask,104000,104002,104004,104015


In [440]:
# 3 create new dataframe with flask id and amount owned
user_items_flasks = user_items_raw[user_items_raw["id"].isin(item_data_flask_rewards["id"])].merge(item_data_flask_rewards)
user_items_flasks.head()

,id,number,cooldown_end_time,flask_name,reward1,reward2,reward3,reward4
0,2558,150,NaT,Toxic Quageye Flask,104005,104006,104013,104025
1,2562,200,NaT,Unbinded Umbu Flask,104000,104010,104017,104027
2,2573,315,NaT,Wicked Froggit Flask,104011,104017,104023,104032
3,2577,220,NaT,Master Maki Flask,104005,104015,104027,104034
4,2591,40,NaT,Harpy Iphelia Flask,104026,104029,104032,104040


In [441]:
# 5 create new dataframe with total amount of shards available per champion (very sql like)
user_items_flasks_melted = user_items_flasks.melt(id_vars=['id', "number"], value_vars=['reward1', 'reward2', 'reward3', 'reward4'], var_name='reward_col', value_name='reward')
print(user_items_flasks_melted.head())

# 
user_items_flasks_grouped = user_items_flasks_melted.groupby(["reward"]).agg({"id":"count", "number":"sum"}).reset_index()
user_items_flasks_grouped.head()

     id  number reward_col  reward
0  2558     150    reward1  104005
1  2562     200    reward1  104000
2  2573     315    reward1  104011
3  2577     220    reward1  104005
4  2591      40    reward1  104026


,reward,id,number
0,104000,2,400
1,104001,2,605
2,104002,2,400
3,104004,3,1360
4,104005,4,820


In [442]:
# create dict with flask ids which give a certain shard
# {'107607': [2596], '104004': [2603, 2648], '104008': [2604, 2658, 2671],...}
source_dict = {}
for i, row in user_items_flasks_melted.iterrows():
    if row["reward"] is not None:
        if row["reward"] in source_dict:
            source_dict[row["reward"]].append(row["id"])
        else:
            source_dict[row["reward"]] = [row["id"]]
            
# create new dataframe from source_dict which this structure: 	            source_0	source_1	source_2
                                                            #    107607	    2596	    NaN	        NaN
                                                            #    104004	    2603	    2648.0	    NaN
                                                            #    104008	    2604	    2658.0	    2671.0
item_data_shard_source = pd.DataFrame.from_dict(source_dict, orient="index", columns=[f"source_{i}" for i in range(1,max([len(val) for val in source_dict.values()])+1)])

# merge previous list with shard sums with shard source (EXLUDING YET UNBUILT CHAMPIONS)
item_data_shards = user_items_flasks_grouped.merge(item_data_shard_source, left_on="reward", right_index=True)
item_data_shards.rename(columns={"id":"num_flasks", "number":"flask_shards"}, inplace=True)
item_data_shards["reward"] = item_data_shards["reward"].astype(int)
item_data_shards.head()

,reward,num_flasks,flask_shards,source_1,source_2,source_3,source_4,source_5,source_6,source_7
0,104000,2,400,2562,2641.0,NaN,NaN,NaN,NaN,NaN
1,104001,2,605,2655,2643.0,NaN,NaN,NaN,NaN,NaN
2,104002,2,400,2624,2673.0,NaN,NaN,NaN,NaN,NaN
3,104004,3,1360,2603,2648.0,2631.0,NaN,NaN,NaN,NaN
4,104005,4,820,2558,2577.0,2672.0,2660.0,NaN,NaN,NaN


In [443]:
# 6 create new dataframe with shard id and number of champion shards in inventory
user_items_champion_shards = []

# append multiple dataframes to lists
for id_min, id_max in zip(champion_shards_id_min, champion_shards_id_max):
    user_items_champion_shards.append(user_items_raw[user_items_raw["id"].between(id_min, id_max)])

# concat dataframes into one
user_items_champion_shards = pd.concat(user_items_champion_shards)
user_items_champion_shards.head()

,id,number,cooldown_end_time
104006,104006,410,NaT
104010,104010,510,NaT
104011,104011,250,NaT
104012,104012,115,NaT
104013,104013,150,NaT


In [444]:
# 7 create new dataframe with unit_id and level of all owned champions (un-crafted might not count yet)
user_units_champions_owned = []

# append multiple dataframes to lists 
for id_min, id_max in zip(champion_id_min, champion_id_max):
    user_units_champions_owned.append(user_units_raw[user_units_raw["unit_id"].between(id_min, id_max)])
    
# concat dataframes into one
user_units_champions_owned = pd.concat(user_units_champions_owned)
    
# keep only the specified columns and drop all others
user_units_champions_owned = user_units_champions_owned.drop(user_units_champions_owned.columns.difference(["unit_id", "level"]), axis=1)
user_units_champions_owned.head()

,unit_id,level
31,4002,9
56,4027,5
77,4030,8
117,4014,8
146,4035,4


In [445]:
# 8
# goal: unit_id | level | shards_sum(owned + available shards) | necessary shards | is_sufficient? | owned shards | flask_shard_sum | flask_count | source_1 | ... | source_n
# ex:   Noctrus | 9     | 965                                  | 900              | True           | 750          | 225             | 2           | 2590... 2631
# fancy print statements?

# convert from shard_it to unit_id
user_items_champion_shards["unit_id"] = user_items_champion_shards["id"] - 100000 # type: ignore

# merge champ level and shards in inventory
full_frame = user_units_champions_owned.merge(user_items_champion_shards, how="right") # type: ignore

# merge shard sources and total form flasks
full_frame = full_frame.merge(item_data_shards, how="left", left_on="id", right_on="reward")

# replace NaN where possible
for col in full_frame:
    if not re.match("source",col): # type: ignore
        full_frame[col] = full_frame[col].fillna(0).astype(int)

# total shards available
full_frame["shards_sum"] = full_frame["number"] + full_frame["flask_shards"]

# shards for next upgrade
def get_cost(level):
    return champion_costs[level]
full_frame["upgrade_cost"] = full_frame["level"].apply(get_cost)

# do you own enough, need flasks or just can't upgrade champ yet
def is_sufficient(owned, potential, cost):
    if owned >= cost:
        return "yes! no flasks necessary!"
    elif owned + potential >= cost:
        return "yes, only with flasks."
    else:
        return f"no, missing {int(cost-potential-owned)}"
full_frame["is_sufficient"] = full_frame.apply(lambda x: is_sufficient(x["number"], x["flask_shards"], x["upgrade_cost"]), axis=1)

full_frame.tail()

,unit_id,level,id,number,cooldown_end_time,reward,num_flasks,flask_shards,source_1,source_2,source_3,source_4,source_5,source_6,source_7,shards_sum,upgrade_cost,is_sufficient
70,7610,6,107610,129,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,129,400,"no, missing 271"
71,7611,0,107611,30,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30,50,"no, missing 20"
72,7614,0,107614,23,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23,50,"no, missing 27"
73,7621,0,107621,23,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23,50,"no, missing 27"
74,7622,0,107622,34,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34,50,"no, missing 16"


In [446]:
user_items_flasks.set_index("id", inplace=True) # only runnable once before issues arise
user_items_flasks.head()

,number,cooldown_end_time,flask_name,reward1,reward2,reward3,reward4
id,,,,,,,
2558,150,NaT,Toxic Quageye Flask,104005,104006,104013,104025
2562,200,NaT,Unbinded Umbu Flask,104000,104010,104017,104027
2573,315,NaT,Wicked Froggit Flask,104011,104017,104023,104032
2577,220,NaT,Master Maki Flask,104005,104015,104027,104034
2591,40,NaT,Harpy Iphelia Flask,104026,104029,104032,104040


In [447]:
# create new dataframe, replacing all ids with readable item names
# TODO: add amount of each flask in table after source

# replace source ids and ignore NaN
final = full_frame.copy()

source_cols = [f"source_{i+1}" for i in range(sum([1 for col in final if re.match("source", col)]))] # type: ignore

# array consisting of the source columns from the dataframe
arr = []
for col in source_cols:
    arr.append(final[col].values)

# {2603.0: ('Tarragon Flask', 200), 2660.0: ('Enchanted Yaritza Flask', 20),...}
replace_dict = {}
for col in arr:
    for row in col:
        if not np.isnan(row):
            if row not in replace_dict:
                replace_dict[row] = (user_items_flasks.loc[row, "flask_name"], user_items_flasks.loc[row, "number"]) # shardbot exception perma fix here

def replace_flask_id(flask_id):
    if not np.isnan(flask_id):
        return replace_dict[flask_id][0], replace_dict[flask_id][1]

for i, col in enumerate(source_cols):
    final[[col,f"amount_{i}"]] = final[col].apply(replace_flask_id).apply(pd.Series) # type: ignore

# end game
# re-order columns
# [('source_1', 'amount_1'), ('source_2', 'amount_2'), ('source_3', 'amount_3')]
# BUG: wrong shard amounts in dictionary
# source_amount_comprehension = [(f"source_{i}", f"amount_{i}") for i in range(1,max([len(val) for val in source_dict.values()])+1)]
# source_amount_comprehension = list(itertools.chain(*source_amount_comprehension))

source_amount_comprehension = [f"source_{i}" for i in range(1,max([len(val) for val in source_dict.values()])+1)]

final = final.reindex(columns=["unit_id","level","shards_sum","upgrade_cost","is_sufficient","number","flask_shards"]+source_amount_comprehension)
final = final.rename(columns={"unit_id":"champ", "number":"shards_owned"})

# for col in source_amount_comprehension[1::2]:
#     final[col] = final[col].fillna(0).astype(int)

# use .xml to replace unit_id by champ name
final.replace(cards_shard_dict, inplace=True)
final.head(50)

,champ,level,shards_sum,upgrade_cost,is_sufficient,shards_owned,flask_shards,source_1,source_2,source_3,source_4,source_5,source_6,source_7
0,LOCO-5000,6,1110,400,yes! no flasks necessary!,410,700,Toxic Quageye Flask,Spring Freya Flask,Salt Wastes Flask,NaN,NaN,NaN,NaN
1,"Columba, Firewing Monk",6,1605,400,yes! no flasks necessary!,510,1095,Unbinded Umbu Flask,Wilderness Predonyx Flask,Frigore Flask,NaN,NaN,NaN,NaN
2,Emerald Amarok,6,775,400,"yes, only with flasks.",250,525,Wicked Froggit Flask,Colossal Moklam Flask,NaN,NaN,NaN,NaN,NaN
3,Cordelia the Twinblade,7,490,600,"no, missing 110",115,375,Protective Yaritza Flask,Duskwillow Rebuilt Flask,NaN,NaN,NaN,NaN,NaN
4,"Teroxis, King of Stones",6,905,400,"yes, only with flasks.",150,755,Tarragon Flask,Toxic Quageye Flask,Mighty Shadomeka Flask,NaN,NaN,NaN,NaN
5,Vindicator Batara,6,900,400,"yes, only with flasks.",70,830,Elaria Holy Flask,Master Maki Flask,Skyhaven Flask,Colossal Moklam Flask,NaN,NaN,NaN
6,Daknak Darkblade,6,1035,400,"yes, only with flasks.",175,860,Golden Flask,Bomber Clockoo Flask,Illustrious Cellia Flask,NaN,NaN,NaN,NaN
7,King Proteus,7,1765,600,yes! no flasks necessary!,650,1115,Wicked Froggit Flask,Pharos Flask,Stormy Seastone Flask,Unbinded Umbu Flask,Duskwillow Flask,NaN,NaN
8,Formos Ebonclaw,7,1550,600,"yes, only with flasks.",350,1200,Beetleton Flask,Prominent Reks Flask,Returned Morty Flask,NaN,NaN,NaN,NaN
9,"Paloma, Spear Knight",5,1175,300,yes! no flasks necessary!,360,815,Wicked Froggit Flask,Conflux Viracocha Flask,Frigore Flask,NaN,NaN,NaN,NaN


In [448]:
# 06.04.23 19:30
# -> flask_extractor.py

In [449]:
# TODO: convert notebook to executable .py
# if __name__ == "__main__":
#     main()